In [30]:
import pandas as pd
from pathlib import Path

In [31]:
here = Path("__file__").parent
data_folder = here / "data"

roster = pd.read_csv(
    data_folder/"roster.csv",
    converters = {
                "NetID": str.lower, 
                "Email Address": str.lower
                },
    usecols = ["Section", "Email Address", "NetID"],
    index_col = "NetID"
)

# dataFrames.
hwcsv = pd.read_csv(
    data_folder/"hw_exam_grades.csv",
    converters = {
                "Email Address": str.lower,
                "SID": str.lower,
                "First Name": str.lower,
                "Last Name": str.lower
    },
    usecols= lambda x: "Submission" not in x,
    index_col = "SID"
)

# [i for i in data_folder.glob("quiz_*_grades.csv")]

quiz_grades = pd.DataFrame()

for file_path in data_folder.glob("quiz_*_grades.csv"):
    quiz_name = " ".join(file_path.stem.title().split("_")[:2])
    # print(file_path)
    quiz = pd.read_csv(
        file_path,
        converters = {
            "Email": str.lower
        },
        usecols=["Email", "Grade"],
        index_col="Email"
    ).rename(columns={"Grade": quiz_name})
    quiz_grades = pd.concat([quiz_grades, quiz], axis=1)



In [32]:
finalData = pd.merge(
    roster,
    hwcsv,
    left_index=True,
    right_index=True
)

finalData = pd.merge(
    finalData,
    quiz_grades,
    left_on = "Email Address",
    right_index = True
    
)

finalData = finalData.fillna(0)

# Average Exam score

for i in range(1,4):
    finalData[f"Exam {i} Score"] = (finalData[f"Exam {i}"] / finalData[f"Exam {i} - Max Points"])
    
homework_score = finalData.filter(regex=r"^Homework \d\d?$", axis = 1)
homework_max = finalData.filter(regex=r"^Homework \d\d? -", axis=1)

sum_hw_scores = homework_score.sum(axis=1)
sum_hw_maxpoints = homework_max.sum(axis=1)

finalData["Total Homework"] = sum_hw_scores / sum_hw_maxpoints

hw_max_renamed = homework_max.set_axis(homework_score.columns, axis=1)

avg_hw_score = ((homework_score / hw_max_renamed).round(2)).sum(axis=1)

percentage_hw = pd.DataFrame()

# avg_hw_score
quiz_grades


,Quiz 5,Quiz 2,Quiz 4,Quiz 1,Quiz 3
Email,,,,,
rachel.dennis@univ.edu,11.0,6,6.0,7.0,7.0
courtney.wolf@univ.edu,9.0,15,7.0,10.0,9.0
aaron.lester@univ.edu,8.0,9,8.0,6.0,10.0
ashley.martin@univ.edu,11.0,7,13.0,5.0,8.0
jessica.garza@univ.edu,10.0,8,11.0,8.0,16.0
...,...,...,...,...,...
tara.johnson@univ.edu,12.0,6,14.0,6.0,10.0
michelle.lee@univ.edu,10.0,13,10.0,9.0,16.0
scott.cervantes@univ.edu,9.0,8,12.0,4.0,9.0
